![Header Image]( ../assets/header_image.png "Header Image")

# Assignment: Localization

Welcome to the assignment **Localization**.

In this assignment, we will use the bag file that has been recorded at the end of the C++ task to evaluate the implemented solution.
In particular, we will walk through the following steps:

- Importing and parsing of the information from the bag file using [rosbags](https://pypi.org/project/rosbags/).
- Converting the imported data into a [pandas](https://pandas.pydata.org/) dataframe.
- Assignment of ground truth poses to the corresponding estimated vehicle poses
- Implementation and visualization of various metrics for the evaluation of the pose estimation.

### Import 
Let's start by importing all the necessary packages for this assignment:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

from rosbags.rosbag2 import Reader
from rosbags.serde import deserialize_cdr

## Introduction to `TrajectoryPoint2D`-Class
In this assignment, we will evaluate the estimated pose of the vehicle. In the first step, we will load the data from the bag file and store the information in a list of `TrajectoryPoint2D`.

An object of class `TrajectoryPoint2D` holds the following members:
 - `x` x-Position of the vehicle in map-frame [m]
 - `y` y-Position of the vehicle in map-frame [m]
 - `psi` yaw-Angle of the vehicle in map-frame [deg]
 - `t` timestamp of pose [s]

Next to the member objects the class offers two relevant functions:
 - `def from_odometry(self, odometry)`
 - `def from_pose(self, pose)`
 
Both functions are called on an `TrajectoryPoint2D` object and set all member variables of this object based on a `nav_msgs::msg::Odometry` respectively a `geometry_msgs::msg::PoseStamped` message.

To make use of this class, we import `TrajectoryPoint2D` in the following:

In [ ]:
from localization.trajectory import TrajectoryPoint2D

## Task: Importing data from a ROS2 Bag-File using [rosbags](https://pypi.org/project/rosbags/)

First of all we need to import the data that we've captured within a ROS2 bag-file.
We will use the [rosbags](https://pypi.org/project/rosbags/) package for this purpose. For further information please refere to the [rosbags documentation](https://ternaris.gitlab.io/rosbags/).

To load the bag file, copy it into the `${REPOSITORY}/bag` directory of this repository.
Afterwards, specify the name of the folder that contains the `*.db3`-file.

In [ ]:
bag_path = '../bag'
### START CODE HERE ###
bag_folder_name = None
### END CODE HERE ###
bag_path += '/'+bag_folder_name

Now we need to finish the implementation of the `read_trajectory` function. This function reads a given bag file and returns a list of `TrajectoryPoint2D` based on a `nav_msgs::msg::Odometry` or a `geometry_msgs::msg::PoseStamped` message. Try to understand how this function works and fill in the gaps.


Your task is to call the functions `from_odometry` respectively `from_point` on a `TrajectoryPoint2D` object and append the resulting point to the trajectory.

In [ ]:
def read_trajectory(bag_path, topic_name):
    """
    Reads a given bag file and returns a list of TrajectoryPoint2D based on a nav_msgs::msg::Odometry or a geometry_msgs::msg::PoseStamped message
    
    Arguments:
    bag_path -- String indicating the location of the bag-file
    topic_name -- String indicating the topic name to derive the trajectory from
    
    Returns:
    trj -- List of TrajectoryPoint2D
    """
    # create and open a rosbag-reader
    with Reader(bag_path) as reader:
        # create an empty list that represents the trajectory
        trj = []
        # get connection that represents the specific topic
        connection = [x for x in reader.connections if x.topic == topic_name]
        # iterate over messages of specific topic
        for connection, timestamp, rawdata in reader.messages(connections=connection):
            # initialize a TrajectoryPoint2D-object
            point = TrajectoryPoint2D()
            # deserialize the rawdata based on the specific msgtype
            msg = deserialize_cdr(rawdata, connection.msgtype)
            if connection.msgtype == 'nav_msgs/msg/Odometry':               
                ### START CODE HERE ###
                # set all members of the TrajectoryPoint2D-object based on a nav_msgs::msg::Odometry
                
                # append the TrajectoryPoint2D-object to trajectory
                
                ### END CODE HERE ###
            elif connection.msgtype=='geometry_msgs/msg/PoseStamped':
                ### START CODE HERE ###
                # set all members of the TrajectoryPoint2D-object based on a geometry_msgs::msg::PoseStamped
                
                # append the TrajectoryPoint2D-object to trajectory
                
                ### END CODE HERE ###
            else:
                # print message if topic contains a message of unsupported type
                print("Message format {} is currently not supported!".format(connection.msgtype))
        # return the trajectory
        return trj

Now that you've implemented `read_trajectory` call this function once for the ground-truth pose on topic `/ground_truth/pose` and for the estimated vehicle pose on topic `/localization/predicted_pose`. Store the resulting trajectory into two specific variables.
##### __Hints__:
- The path to the bag file is stored in the variable `bag_path`.

In [ ]:
### START CODE HERE ###


### END CODE HERE ###

## Task: Creating a pandas DataFrame from both trajectories

Now that we've succesfully imported the trajectory data, we will convert these into a [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

At first, we will create two DataFrame: one for the ground truth trajectory and one for the estimated trajectory. Each DataFrame contains four colums `t`, `x`, `y` and `psi`. In our case, we use list comprehension to store all these trajectory variables into the DataFrame.

Your task is to replace all `None` placeholders within the code cell below.
##### __Hints__:
- The `None` placeholders refer to the specific trajectory we would like to derive the information from.

In [ ]:
### START CODE HERE ###
# Ground-Truth-Trajectory-Data-Frame
df_gt = pd.DataFrame({'t': [point.t for point in None], 'x': [point.x for point in None], 'y': [point.y for point in None], 'psi': [point.psi for point in None]})
# Estimated-Trajectory-Data-Frame
df_est = pd.DataFrame({'t': [point.t for point in None], 'x': [point.x for point in None], 'y': [point.y for point in None], 'psi': [point.psi for point in None]})
### END CODE HERE ###

Now, we have two data-frames, but to compare the trajectories we want the individual ground truth and estimated poses from both DataFrames in only one DataFrame. To achieve this, we combine the two DataFrames into one. The goal is to assign a corresponding ground truth pose to each estimated pose. Since the ground truth trajectory was published with a higher frequency (20Hz) compared to the estimated trajectory (10Hz), the temporal error should be maximal at half the period of the publication frequency of the ground truth pose.

To merge both DataFrames, we will use the function `merge_asof` from pandas. This function allows to merge two DataFrames based on a distance metric of a defined field. In our case, we use the time difference between the poses as a metric. Your task is to choose the `tolerance` for matching two poses. Replace the `None` placeholder in the code cell below.
##### __Hints__:
- The ground truth pose is published with a frequency of 20 Hz. The period between two messages is therefore 0.05 seconds.

In [ ]:
### START CODE HERE ###
df = pd.merge_asof(df_est, df_gt, on='t', tolerance=None, direction='nearest', suffixes=['_estimate', '_ground_truth'])
### END CODE HERE ###

## Subjective evaluation of trajectories

A naive way to evaluate two trajectories is to visualize them in a 2D plot and compare them subjectively. Execute the code cell below to generate a plot.

In [ ]:
# Create an overview plot that visualizes both trajectories
fig, ax = plt.subplots(1,1,figsize=(20, 8))
# Plot ground-truth trajectory
df.plot(x='x_ground_truth', y='y_ground_truth', ax=ax)
# Plot estimated trajectory
df.plot(x='x_estimate', y='y_estimate', ax=ax)
# Set equal axes
ax.axis('equal')
# Hide axis labels
ax.set_xlabel('')
ax.set_ylabel('')
# Set legend labels
ax.legend(['Ground Truth Trajectory', 'Estimated Trajectory'])
# Create a second plot with detailed views
fig, ax = plt.subplots(1,3,figsize=(20, 8))
# Plot trajectories on specific subplots
df.plot(x='x_ground_truth', y='y_ground_truth', ax=ax[0], marker='.', linestyle='none', legend=None)
df.plot(x='x_estimate', y='y_estimate', ax=ax[0], marker='.', linestyle='none', legend=None)
df.plot(x='x_ground_truth', y='y_ground_truth', ax=ax[1], marker='.', linestyle='none', legend=None)
df.plot(x='x_estimate', y='y_estimate', ax=ax[1], marker='.', linestyle='none', legend=None)
df.plot(x='x_ground_truth', y='y_ground_truth', ax=ax[2], marker='.', linestyle='none', legend=None)
df.plot(x='x_estimate', y='y_estimate', ax=ax[2], marker='.', linestyle='none', legend=None)
# Set axes limits for detail views
ax[0].set_xlim(2370, 2410)
ax[0].set_ylim(470, 510)
ax[1].set_xlim(2370, 2410)
ax[1].set_ylim(620, 660)
ax[2].set_xlim(2500, 2520)
ax[2].set_ylim(640, 660)
# Hide axis labels
for axis in ax:
    axis.set_xlabel('')
    axis.set_ylabel('') 
plt.show()

At first glance, the comparison of the trajectories in the overview view looks good. Only in the turns, larger deviations are observable. These deviations are mainly due to the estimation of the vehicle yaw angle from two subsequent GNSS positions.
However, a look at the detailed views also shows that there are poor results in the pose estimates when driving straight (in the area of $y \approx 650 m$). In this area, LiDAR odometry could not provide sufficient motion estimates due to insufficient features. You should take into account that this is only noticeable because of the visualization of the actual measuring points in the plot. It therefore makes sense to visualize the actual measuring points at least in addition to the connecting lines.

### Task: Visualizing the yaw-angle

In the following, we will also take a brief look at the yaw angle, since this is not explicitly shown in the 2D plot.
To do so, please fill in the gaps in the code-cell below.

In this plot, we want to plot the yaw angle over time. Two plots are to be displayed. On the one hand the ground truth yaw angle and on the other hand the estimated yaw angle.

##### __Hints__:
- The best way to get inspired is to look at the code example where we have plotted the trajectories x over y: `df.plot(x='x_ground_truth', y='y_ground_truth', ax=ax)`
- Time is denoted as `'t'` in the DataFrame
- The yaw angle is denoted as `'psi_<ground_truth/estimate>'` in the DataFrame

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20, 4))
### START CODE HERE ###


### END CODE HERE ###
# Set legend labels
ax.legend(['Ground Truth Trajectory', 'Estimated Trajectory'])
# Set axes limits
ax.set_xlim(310, 360)
ax.set_ylim(-200, 200)
plt.show()

The plot shows that the vehicle is initially aligned with a yaw angle of approx. 180°. Since the yaw angle is defined between -180° and 180° here, there are discontinuities that can be seen in the plot. However, these are not really dramatic in this case. It gets interesting at the point where the vehicle stops at a stop sign. Here, the estimated yaw angle jumps even though the vehicle is stationary. This behavior is also due to the estimation based on two sequential GNSS positions, since these are very close to each other, but can slightly deviate from each other due to measurement errors, resulting in arbitrary yaw angle estimates.

With this simple analysis, we have already identified a major weakness of the implementation. This could be counteracted, for example, by using an additional sensor for direct measurement of the vehicle orientation. An example of such a sensor is a compass.

In the following, we will calculate some metrics to numerically describe the quality of the implemented approach.

## Task: Calculation and investigation of metrics for the evaluation of localization quality

We will start with the calculation of the most basic metrics: The distance error between ground truth and estimate in `x`, `y` and `psi`.

Pandas allows us to add new fields to the DataFrame based on calculations in a very intuitive way. Use the example below to add the corresponding columns named `dx`, `dy` and `dpsi` to the DataFrame.

#### Example:
`example_data_frame['C'] = example_data_frame['A'] + example_data_frame['B']`
- In this example `A` and `B` are existing fields in the DataFrame, the new field with name `C` will be added and represents the sum of `A` and `B`.

In [ ]:
### START CODE HERE ###



### END CODE HERE ###

Now that we have calculated the deviations in x, y and psi, we can examine them in more detail. Obviously, we could plot the values over time again, but the information we would get from this is relatively similar to the 2D plot we looked at before, where these absolute deviations are visually represented. Alternatively, we will plot the deviations in the form of boxplots.

Execute the code cell below to generate boxplots for `dx`, `dy` and `dpsi`.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20, 4))
df.boxplot(column='dx', ax=ax[0], whis=[0, 100])
df.boxplot(column='dy', ax=ax[1], whis=[0, 100])
df.boxplot(column='dpsi', ax=ax[2], whis=[0, 100])
ax[0].set_ylim(-2.5, 2.5)
ax[1].set_ylim(-0.15, 0.15)
ax[2].set_ylim(-0.1, 0.1)

plt.show()

The box in this plot represents the range of values in which 50% of the data fall. The lower boundary of the box is also called the lower quartile and describes the value that is greater than or equal to 25% of the smallest values in the data set. The upper quartile corresponds to the value that is greater or equal than 75% of the smallest values. In other words, 25% of all data points are greater than this value.

Inside the box, there is a green line that describes the median. The median divides the data set into two classes containing the largest and the smallest values, i.e. 50% of all values are above and 50% of all values are below the median.

In addition to the box and the median, the so-called whiskers are also visualized. In this case, the whiskers describe the minimum or maximum value in the data set. Note that depending on the axis scaling, not all whiskers can be recognized.

The interpretation of `dx` and `dy` is not very meaningful from the technical point of view as they indicate the deviations of the position in the map frame. Related to the vehicle, the evaluation in longitudinal and lateral direction is of particular importance. The longitudinal deviation describes the deviation along the longitudinal axis of the vehicle, and the lateral deviation describes the deviation perpendicular to the longitudinal axis of the vehicle.

We will determine these vehicle centered translational deviations in the following. Before we do so, we can briefly discuss the boxplot for the yaw angle deviation (`dpsi`). The median deviation of the yaw angle is just under a quarter of a degree. It is noticeable that the upper quartile is greater in magnitude than the lower quartile. In other words, there is a greater deviation to the left than to the right. A possible explanation for this may be that the vehicle makes two right-hand turns, resulting in deviations in the vehicle yaw angle to the left. We have already explained the large minimum and maximum values in the above plot of the deviation of `psi` over time. We came to the conclusion that the estimation of the yaw angle from two sequential gnss poses is mainly responsible for this. 

Now, lets calculate the longitudinal and lateral deviation relative to the `ground_truth` pose of the vehicle. Since `dx` and `dy` are given with respect to the map frame, we need to rotate these values with respect of the yaw angle (`psi_ground_truth`). The transformation can be implemented using the following equations:

$d_{lon} = dx * cos(\psi) + dy * sin(\psi)$

$d_{lat} = -dx * sin(\psi) + dy * cos(\psi)$

You can apply numpy's trigonometric functions on coloums of your pandas DataFrame:
#### Example:
`example_data_frame['sin_x'] = np.sin(example_data_frame['x'])`
- In this example `x` is an existing field in the DataFrame. The new field with name `sin_x` will be added representing $sin(x)$ for each value of `x` in the data-frame.

Now complete the cell below to calculate and add `dlon` and `dlat` to the data-frame `df`.

##### __Hints__:
- `psi_ground_truth` is stored in degrees within the DataFrame while `np.sin` expects the input to be radians
- The constant $\pi$ can be accessed through `np.pi` 

In [ ]:
### START CODE HERE ###



### END CODE HERE ###

Now that we have calculated `dlon` and `dlat`, we can visualize them again in the form of a boxplot. Execute the code cell below to generate the plots.

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20, 4))
df.boxplot(column='dlon', ax=ax[0], whis=[0, 100])
df.boxplot(column='dlat', ax=ax[1], whis=[0, 100])
df.boxplot(column='dlat', ax=ax[2], whis=[0, 100])
ax[1].set_ylim(-0.25, 0.25)
plt.show()

The left graph shows the boxplot for `dlon` while the other two graphs show the boxplot for `dlat`. In each case, only the y-axis is scaled differently in order to better recognize the box and the minimum and maximum values.

You can see that the error in the lateral direction is distributed on significantly smaller scales than in the longitudinal direction. This may be due to the fact that the vehicle performs only few maneuvers, which challenge the localization in the context of the lateral direction.

If you compare the boxplots of `dlon` and `dlat` with those of `dx` and `dy`, you can see a certain correlation. This is mainly due to the fact that in the scenario we investigated, the vehicle moves in the direction of the x-axis of the map coordinate system for quite a long time, which results in a certain correlation. In reality, however, this is usually not the case, which is why it makes sense to evaluate in the lateral and longitudinal directions.

Finally, we investigate to how the lateral and longitudinal errors are distributed spatially. For this purpose, the ground trouth trajectory is visualized in a 2D plot and the longitudinal and lateral errors are represented by color coding of the points.

Execute the code cells below to generate the plots.

In [ ]:
colormap = plt.get_cmap('viridis')
normalize = Normalize(vmin=0, vmax=2.0)
scalar_mappable = ScalarMappable(cmap=colormap, norm=normalize)
colors = scalar_mappable.to_rgba(np.abs(df['dlon']))
fig, ax = plt.subplots(1,1,figsize=(20, 8))
ax.axis('equal')
plt.scatter(df['x_ground_truth'], df['y_ground_truth'], c=colors, marker='.')
colorbar = plt.colorbar(scalar_mappable)
colorbar.set_label('dlon')
plt.show()

The plot above shows how the longitudinal error is distributed. It becomes visible that the error increases especially in the last section because of the failing LiDAR odometry. It is also interesting that in the middle section, where the vehicle moves in the y-direction of the map, larger errors occur. This is possibly due to the speed of the vehicle in this section, which can be seen from the larger distances between the points.

In [ ]:
colormap = plt.get_cmap('viridis')
normalize = Normalize(vmin=0, vmax=0.2)
scalar_mappable = ScalarMappable(cmap=colormap, norm=normalize)
colors = scalar_mappable.to_rgba(np.abs(df['dlat']))
fig, ax = plt.subplots(1,1,figsize=(20, 8))
ax.axis('equal')
plt.scatter(df['x_ground_truth'], df['y_ground_truth'], c=colors, marker='.')
colorbar = plt.colorbar(scalar_mappable)
colorbar.set_label('lat')
plt.show()

The lateral error is especially increased in the areas where the vehicle passes through a turn. As described before, this is due to the estimation of the vehicle yaw angle.

## Wrap Up

Congratulations, you have successfully completed the Notebook Exercise regarding the section of vehicle localization!

In this task you learned how to ...

- ... import and parse information from a ROS 2 bag file using [rosbags](https://pypi.org/project/rosbags/).
- ... convert the imported data into a [pandas](https://pandas.pydata.org/) DataFrame.
- ... assign the ground-truth poses to the corresponding vehicle poses estimated by the implemented localization stack.
- ... implement and display various metrics for the evaluation of the pose estimation.

## License


MIT License

Copyright 2023 Institute for Automotive Engineering of RWTH Aachen University.